# Numerical methods for non linear ODEs with initial values

Only a few non-linear ODEs have analytical solutions. To evaluate the solution of such ODEs one may resolve to numerical methods. We start by discussing the simplest case of a first order initial value problem (IVP) of the form

$du(t)/dt = f(t,u)$

with initial condition

$u(t_0) = u_0$

So the solution of the ODE is the function u and the function f that makes the ODE non-homogeneous is, in principle, a non-linear function. The solution depends on time and the function f depends on both time and u, in the most general case. The fundamental theorem of calculus applied to the ODE above implies that

[1]  $\quad u(t_{i+1}) = u(t_i) + \int_t^{t_{i+1}} f(t',u(t'))dt' $

 # Runge-Kutta 4

This code permits to evaluate an integral with a numerical techinque called Runge-Kutta of order 4. It leads to an error with respect to the exact value of the integral that is $\Delta x = x^4$.

This code need NumPy and the definition of a customary function that is the function to integrate. For this example we take the simplest case of a function f of a single variable x.

In [ ]:
import numpy as np

In [ ]:
def f(x):
  return 1/(x**2)

## 1.1 Euler method

### 1.1.1 Function f does not depend on the solution

Start with something simple: the Euler method. You integrate the function f(t) from a starting point a to an ending point b. The approximate value that you get is the value of the function at the starting point multiplied by the lenght (b-a). To make it work well, we divide the range [a:b] into linearly spaced intervals and evaluate the integral for each step. The iteration of the evaluation for each step leads to a small integral. The final integral is made by the summation of all the small integrals.

$ \int_{x_i}^{x_{i+1}} f(x)dx \approx hf(x_0) + hf(x_1) + \ldots + hf(x_{n-2})  $

We may define a new function that operates the Euler method. We need to know the starting point of the function, meaning the value of the y in the starting point a.

We should build a grid of x points that are our $t_i$. This is done by dividing the interval (b-a) into (n-1) parts. Each step will then have a length of

$ h = (b-a)/(n-1) $

Then we build the grid in the form of a NUMPY ARRAY.

We decide to use NumPy arrays to build the values of f(x,y) too. So we have to define a vector of proper length, meaning (n-1)+1 = n points.





In [ ]:
def euler(f, a, b, n):
  h = (b-a)/(n-1)
  xs = a + np.arange(n-1)*h
  y = np.zeros(n-1)

  for i,x in enumerate(xs): #enumerate returns an indexed list of (0,xs[0]),(1,xs[1])...
   y[i] = h*f(x)
    
  return np.sum(y)

I now compute the integral of f from 1 to 2 taking different numbers of steps. The expected value of the integral is

$ \int_1^2 1/x^2 dx = 1/2 $

Note that this function has a a polar singularity of order 2 in x=0, so I may not evaluate the integral starting or ending at 0.

In [ ]:
euler(f, 1, 2, 10)

0.5434622001292393

In [ ]:
euler(f, 1, 2, 100)

0.5038027578858124

In [ ]:
euler(f, 1, 2, 1000)

0.500375521500781

### Note: here I have exploited the method to evaluate an integral. In general, the aim of the method goes through the evaluation of an integral only to calculate the value of the solution $u(t)$ of the ODE in a lot of points (and so to draw it).

### 1.1.2 Function f depends on the solution

Now we conside the most general case in which the function f is an arbitrary function of both the solution and the time parameter t, so $f = f(t, u(t))$.

The left hand side of the fundamental theorem of calculus [1] contains an integral that is approximated, with the Euler's method, with

$ \int \ldots \approx f(t_i, u(t_i))(t_{i+1}-t_i)$

Here the main problem is that the function $u(t_i)$ is not know anywhere alse than the initial point $u_0$, so we must iterate and construct the function u for each point of the grid of the $t_i$.

In [ ]:
def f_t(t,u):
  return u/(t**2+1)

In [ ]:
def euler_t(f, a, b, n, uinit): #uinit = u(t_i)
  h = (b-a)/(n-1)
  ts = a + np.arange(n)*h #grid in time
  us = np.zeros(n) #grid in u(t), array of same size of xs
  #us will be populated in loop below
  
  u = uinit
  for i,t in enumerate(ts): #enumerate returns an indexed list of (0,xs[0]),(1,xs[1])...
   us[i] = u
   u += h*f(t,u)
    
  return ts, us

I don't need to compute the total integral here because I only want the evolution in time (in each point of the time grid) of the solution of the ODE.

In [ ]:
intestation = "Time grid | Solution \n"

In [ ]:
ts,us = euler_t(f_t, 1, 2, 11, 1.5)
resultEuler = np.column_stack((ts,us))
resultEuler = np.around(resultEuler, decimals = 3)
print(intestation,"\n",resultEuler)

Time grid | Solution 
 
 [[1.    1.5  ]
 [1.1   1.575]
 [1.2   1.646]
 [1.3   1.714]
 [1.4   1.777]
 [1.5   1.837]
 [1.6   1.894]
 [1.7   1.947]
 [1.8   1.997]
 [1.9   2.044]
 [2.    2.089]]


## 1.2 Runge-Kutta order 4

This method is based on the Simpson's rule, that approximate each integral with

$ \int_a^b f(x) dx = (b-a)/6 * [f(a) + 4f((a+b)/2) + f(b)] $

Each sub-integral evaluated for a single step h is equal to

$ u(t_{i+1}) = u(t_i) + \Delta t /6 * [k_1 + 2k_2 + 2k_3 + k_4]  $

where

$ k_0 = f(t_i, u_i) $

$k_1 = f(t_1 + \Delta t /2, u_i + k_0 \Delta t/2)$

$k_2 = f(t_i + \Delta t/2, u_i + k_1 \Delta t/2)$

$k_3 = f(t_{i+1}, u_i + k_2 \Delta t)$



In [ ]:
def rk4(f,a,b,n,uinit):
  h = (b-a)/(n-1)
  ts = a + np.arange(n)*h
  us = np.zeros(n)
  u = uinit

  for i,t in enumerate(ts):
    us[i] = u
    k0 = f(t,u)
    k1 = f(t+h/2, u + k0*h/2)
    k2 = f(t + h/2, u + k1*h/2)
    k3 = f(t + h, u + k2*h)
    u = u + h/6*(k0+2*k1+2*k2+k3)

  result = np.column_stack((ts, us))
  return result

In [ ]:
resultRK4 = rk4(f_t, 1, 2, 11, 1.5)
resultRK4 = np.round(resultRK4, decimals = 3)
print(intestation, resultRK4)

Time grid | Solution 
 [[1.    1.5  ]
 [1.1   1.573]
 [1.2   1.642]
 [1.3   1.708]
 [1.4   1.769]
 [1.5   1.827]
 [1.6   1.882]
 [1.7   1.933]
 [1.8   1.981]
 [1.9   2.027]
 [2.    2.069]]


In [ ]:
comparison_Eu_RG4 = np.column_stack((resultEuler[:],resultRK4[:,1]))
print(intestation, comparison_Eu_RG4)

Time grid | Solution 
 [[1.    1.5   1.5  ]
 [1.1   1.575 1.573]
 [1.2   1.646 1.642]
 [1.3   1.714 1.708]
 [1.4   1.777 1.769]
 [1.5   1.837 1.827]
 [1.6   1.894 1.882]
 [1.7   1.947 1.933]
 [1.8   1.997 1.981]
 [1.9   2.044 2.027]
 [2.    2.089 2.069]]
